# Human, Animal, and Vehicle Intrusion Detection System

In [4]:
# !pip install ultralytics
# !pip install ultralytics==8.0.100
# !pip install playsound==1.2.2  # Older version works best


In [1]:
# !ls 
# !rm -r  yolo*

In [2]:
from ultralytics import YOLO
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import time

model = YOLO('yolov8n.pt')  # Will redownload a fresh copy
# model = YOLO('yolov8n/data.pkl')

/home/akashs/.local/lib/python3.8/site-packages/ultralytics/nn/tasks.py:377: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file 

In [3]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [4]:
import cv2

# Access default webcam (0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Cannot open webcam")
else:
    print("✅ Webcam accessed successfully")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    cv2.imshow('Webcam Feed (Press q to quit)', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close window
cap.release()
cv2.destroyAllWindows()


✅ Webcam accessed successfully


In [1]:
import cv2
import time
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8n.pt")
names = model.names

# Classes to detect: All animals (except birds) + all vehicles
allowed_labels = [
    'person', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe',  # ← all COCO animals except 'bird'
    'bicycle', 'car', 'motorcycle', 'bus', 'truck'
]

# Access default webcam
cap = cv2.VideoCapture(0)

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_filename = f"recorded_{time.strftime('%Y%m%d_%H%M%S')}.avi"
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_filename, fourcc, 20.0, (frame_width, frame_height))

img_counter = 0
print("Webcam started. Press 's' to save image, 'q' to quit.")

from playsound import playsound

alert_triggered = False  # Control to avoid repeating alerts every frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    boxes = results[0].boxes
    detection_this_frame = False

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        label_name = names[cls]
        label = f"{label_name}: {conf:.2f}"

        if label_name in allowed_labels:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            print(f"🟢 Detected: {label} at ({x1}, {y1}, {x2}, {y2})")

            # Trigger alert only if person is detected
            if label_name == 'person':
                detection_this_frame = True


    # 🔔 Alert: Play sound + Save snapshot (on person detection)
    if detection_this_frame and not alert_triggered:
        print("🚨 Alert: Person detected!")
        playsound("chime_alert.mp3")  # Play sound

        # Save snapshot
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        alert_img = f"alert_person_{timestamp}.jpg"
        cv2.imwrite(alert_img, frame)
        print(f"📸 Alert snapshot saved: {alert_img}")

        alert_triggered = True
    elif not detection_this_frame:
        alert_triggered = False  # Reset if person not in frame

    out.write(frame)
    cv2.imshow('Webcam Feed with Drawing', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    elif key & 0xFF == ord('s'):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        img_filename = f"snapshot_{timestamp}.jpg"
        cv2.imwrite(img_filename, frame)
        print(f"Snapshot saved: {img_filename}")
        img_counter += 1


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Recorded video saved as: {output_video_filename}")


/home/akashs/.local/lib/python3.8/site-packages/ultralytics/nn/tasks.py:377: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file 

Webcam started. Press 's' to save image, 'q' to quit.


Invalid MIT-MAGIC-COOKIE-1 key/home/akashs/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0

0: 480x640 1 cat, 233.3ms
Speed: 11.3ms preprocess, 233.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


🟢 Detected: cat: 0.39 at (182, 123, 383, 379)



0: 480x640 1 cat, 320.8ms
Speed: 4.2ms preprocess, 320.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 1 cat, 143.4ms
Speed: 3.1ms preprocess, 143.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


...

🟢 Detected: person: 0.25 at (76, 39, 567, 480)
🚨 Alert: Person detected!



0: 480x640 1 cat, 132.4ms
Speed: 3.8ms preprocess, 132.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)



📸 Alert snapshot saved: alert_person_20250618_015233.jpg
🟢 Detected: cat: 0.33 at (16, 32, 575, 479)


0: 480x640 1 cat, 1 toilet, 140.4ms
Speed: 4.0ms preprocess, 140.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 cat, 179.6ms
Speed: 3.8ms preprocess, 179.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


🟢 Detected: cat: 0.41 at (81, 72, 546, 479)
🟢 Detected: cat: 0.36 at (43, 22, 544, 478)
Recorded video saved as: recorded_20250618_015050.avi


# Alert

In [1]:
import cv2
import time
from ultralytics import YOLO
from playsound import playsound
import smtplib
import ssl
from email.message import EmailMessage

# -------------------------------
# 🔧 CONFIGURE EMAIL ALERT
# -------------------------------
def send_email_alert(subject, body, to_email, image_path=None):
    sender_email = "akashsipcs@gmail.com"       # ✅ your Gmail
    app_password = "zeyw jake wutq ppsd"           # ✅ your 16-character App Password

    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = to_email
    msg.set_content(body)

    if image_path:
        with open(image_path, "rb") as img:
            img_data = img.read()
            msg.add_attachment(img_data, maintype="image", subtype="jpeg", filename=image_path)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, app_password)
        server.send_message(msg)

    print("📧 Email alert sent!")


# -------------------------------
# 🎯 OBJECT DETECTION SETUP
# -------------------------------
model = YOLO("yolov8n.pt")
names = model.names

allowed_labels = [
    'person', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'bicycle', 'car', 'motorcycle', 'bus', 'truck'
]

cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_filename = f"recorded_{time.strftime('%Y%m%d_%H%M%S')}.avi"
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_filename, fourcc, 20.0, (frame_width, frame_height))

img_counter = 0
alert_triggered_person = False
alert_triggered_general = False

print("Webcam started. Press 's' to save image, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    boxes = results[0].boxes

    detected_labels = []
    person_detected_this_frame = False

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        label_name = names[cls]
        label = f"{label_name}: {conf:.2f}"

        if label_name in allowed_labels:
            detected_labels.append(label_name)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            print(f"🟢 Detected: {label} at ({x1}, {y1}, {x2}, {y2})")

            if label_name == 'person':
                person_detected_this_frame = True

    # 🔔 Play chime if any allowed object detected (once per new detection frame)
    if detected_labels and not alert_triggered_general:
        playsound("chime_alert.mp3")
        alert_triggered_general = True
    elif not detected_labels:
        alert_triggered_general = False

    # 📧 Send email if person detected
    if person_detected_this_frame and not alert_triggered_person:
        print("🚨 Alert: Person detected!")
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        alert_img = f"alert_person_{timestamp}.jpg"
        cv2.imwrite(alert_img, frame)
        print(f"📸 Alert snapshot saved: {alert_img}")

        send_email_alert(
            subject="🚨 Person Intrusion Detected!",
            body="A person was detected on your camera feed.",
            to_email="akashsipcs@gmail.com",
            image_path=alert_img
        )
        alert_triggered_person = True
    elif not person_detected_this_frame:
        alert_triggered_person = False

    out.write(frame)
    cv2.imshow('Webcam Feed with Drawing', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    elif key & 0xFF == ord('s'):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        img_filename = f"snapshot_{timestamp}.jpg"
        cv2.imwrite(img_filename, frame)
        print(f"Snapshot saved: {img_filename}")
        img_counter += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Recorded video saved as: {output_video_filename}")


/home/akashs/.local/lib/python3.8/site-packages/ultralytics/nn/tasks.py:377: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file 

Webcam started. Press 's' to save image, 'q' to quit.


Invalid MIT-MAGIC-COOKIE-1 key/home/akashs/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0

0: 480x640 1 cat, 211.6ms
Speed: 8.5ms preprocess, 211.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


🟢 Detected: cat: 0.51 at (184, 72, 469, 408)



0: 480x640 1 cat, 296.0ms
Speed: 4.9ms preprocess, 296.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)



...

🟢 Detected: person: 0.73 at (0, 80, 535, 479)
🟢 Detected: person: 0.50 at (156, 81, 548, 478)


0: 480x640 2 persons, 227.0ms
Speed: 3.4ms preprocess, 227.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 toilet, 156.3ms
Speed: 4.6ms preprocess, 156.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)



🟢 Detected: person: 0.42 at (101, 75, 542, 479)
🟢 Detected: person: 0.41 at (0, 80, 484, 479)


0: 480x640 (no detections), 235.8ms
Speed: 3.8ms preprocess, 235.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 260.3ms
Speed: 4.3ms preprocess, 260.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


🟢 Detected: person: 0.37 at (143, 81, 545, 479)
🚨 Alert: Person detected!
📸 Alert snapshot saved: alert_person_20250618_022416.jpg
📧 Email alert sent!
Recorded video saved as: recorded_20250618_022316.avi
